In [104]:
from nltk import FreqDist
from nltk.corpus.reader.api import CorpusReader
from nltk.data import LazyLoader
from nltk.tokenize import TreebankWordTokenizer
from nltk.util import AbstractLazySequence, LazyMap, LazyConcatenation
from pymongo import MongoClient
import bs4

import pandas as pd
import os
import json
from pymystem3 import Mystem
import pymorphy2
from pprint import pprint
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn.base import BaseEstimator, TransformerMixin

In [79]:
raw_data_path = os.path.join("..", "..", "data", "raw")
processed_data_path = os.path.join("..", "..", "data", "processed")
models_path = os.path.join("..", "..", "models")
experiments_path = os.path.join("..", "..", "experiments")

In [ ]:
class MongoDBLazySequence(AbstractLazySequence):
    def __init__(self, host='localhost', port=27017, db='test',
        collection='corpus', field='text'):
        self.conn = MongoClient(host, port)
        self.collection = self.conn[db][collection]
        self.field = field
    
    def __len__(self):
        return self.collection.count_documents({})
    
    def iterate_from(self, start):
        f = lambda d: d.get(self.field, '')
        return iter(LazyMap(f, self.collection.find(projection={self.field: 1}, skip=start)))


class MongoDBCorpusReader:
    def __init__(self, word_tokenizer=TreebankWordTokenizer(),
        sent_tokenizer=LazyLoader('tokenizers/punkt/PY3/english.pickle'),**kwargs):
        self._seq = MongoDBLazySequence(**kwargs)
        self._word_tokenize = word_tokenizer.tokenize
        self._sent_tokenize = sent_tokenizer.tokenize

    def text(self):
        return self._seq
    
    def words(self):
        return LazyConcatenation(LazyMap(self._word_tokenize, self.text()))
    
    def sents(self):
        return LazyConcatenation(LazyMap(self._sent_tokenize, self.text()))

In [51]:
corpus_reader = MongodbCorpusReader(('localhost', 27017), "publicru_test", "documents_collection")

In [52]:
len(corpus_reader)

582052

In [53]:
corpus_reader.size()

3404

In [62]:
reader = MongoDBCorpusReader(db='publicru_test', collection='documents_collection', field='body')

In [63]:
reader.sents()

['<body><p><b>Мы взяли две компании с сопоставимой выручкой и оценили влияние курса рубля на их финансы.', 'Первый участник - это производитель удобрений - «ФосАгро», который большую часть выручки получает от экспорта.', ...]

In [64]:
reader.sents()

['<body><p><b>Мы взяли две компании с сопоставимой выручкой и оценили влияние курса рубля на их финансы.', 'Первый участник - это производитель удобрений - «ФосАгро», который большую часть выручки получает от экспорта.', ...]

In [65]:
reader.words()

['<', 'body', '>', '<', 'p', '>', '<', 'b', '>', 'Мы', ...]

In [66]:
reader.text()

['<body><p><b>Мы взяли две компании с сопоставимой выручкой и оценили влияние курса рубля на их финансы. Первый участник - это производитель удобрений - «ФосАгро», который большую часть выручки получает от экспорта. Второй участник ориентирован на внутренний рынок - это компания «М.видео», продающая электронику и технику. В качестве начала отчета возьмем 2013 год, когда курс рубля к доллару оставался стабильным и находился в диапазоне 30 - 33,5.</b></p>\n<p><img alt="" src="325/1.jpg" title=""/></p>\n</body>', '<body><p><b>Индексы потребительских цен на товары и услуги по Российской Федерации в апреле - июне 2016 г.:</b></p>\n<p><img alt="" src="21225/1.jpg" title=""/></p>\n<p><b>За ценами на продукты следите на сайте www.rg.ru/sujet/3131</b></p>\n</body>', ...]

In [68]:
len(reader.text())

582052

In [ ]:
class MongoDBLazySequence(AbstractLazySequence):
    def __init__(self, host='localhost', port=27017, database='test', collection='corpus', field='text'):
        self.conn = MongoClient(host, port)
        self.collection = self.conn[db][collection]
        self.field = field
    
    def __len__(self):
        return self.collection.count_documents({})
    
    def iterate_from(self, start):
        f = lambda d: d.get(self.field, '')
        return iter(LazyMap(f, self.collection.find(projection={self.field: 1}, skip=start)))

class MongodbCorpusReader(CorpusReader):
    """
    MongodbCorpusReader работает с корпусом текстов в HTML формате и 
    предоставляет инструменты предварительной обработки 
    """
    
    def __init__(self, connections, database, collection):
        self._seq = MongoDBLazySequence(**kwargs)
        self.client = MongoClient(*connections)
        self.db = self.client[database]
        self.collection = self.db[collection]

    def __len__(self):
        return self.collection.count_documents({})
    
    def size(self):
        return self.db.command("collstats", self.collection.name, scale=1024*1024)["storageSize"]
            
    def paras():
        paras_tags = ["h1", "h2", "h3", "h4", "h5", "h6", "p", "li"]
        for html in self.docs:
            soup = bs4.BeautifulSoup(html, "lxml")
            for element in soup.find_all(paras_tags)
                yield element.text
            soup.decompose()
            
    def describe(self):      
        counts = FreqDist()
        tokens = FreqDist()
        
        for para in self.paras():
            count["paras"] += 1
            
            for sent in para:
                count["sents"] += 1
                
                for word, tag in sent:
                    counts["words"] += 1
                    tokens[word] += 1
                    
        n_docs = None
        n_topics = None
        
        return {
            "n_docs": n_docs,
            "n_topics": n_topics,
            "paras": counts["paras"],
            "sents": counts["sents"],
            "words": counts["words"],
            "vocab": len(tokens),
            "lexdiv": float(counts["words"]) / float(len(tokens)),
            "ppdoc": float(counts["paras"]) / float(n_docs),
            "sppar": float(counts["sents"]) / float(counts["paras"]),
        }
                

In [ ]:
class MongoDBLazySequence(AbstractLazySequence):
    def __init__(self, host='localhost', port=27017, db='test', collection='corpus', field='text'):
        self.conn = MongoClient(host, port)
        self.collection = self.conn[db][collection]
        self.field = field
    
    def __len__(self):
        return self.collection.count_documents({})
    
    def iterate_from(self, start):
        f = lambda d: d.get(self.field, '')
        return iter(LazyMap(f, self.collection.find(projection={self.field: 1}, skip=start)))


class MongoDBCorpusReader:
    def __init__(self, word_tokenizer=TreebankWordTokenizer(),
        sent_tokenizer=LazyLoader('tokenizers/punkt/PY3/english.pickle'),**kwargs):
        self._seq = MongoDBLazySequence(**kwargs)
        self._word_tokenize = word_tokenizer.tokenize
        self._sent_tokenize = sent_tokenizer.tokenize

    def text(self):
        return self._seq
    
    def words(self):
        return LazyConcatenation(LazyMap(self._word_tokenize, self.text()))
    
    def sents(self):
        return LazyConcatenation(LazyMap(self._sent_tokenize, self.text()))

In [80]:
df_processed_documents =  pd.read_csv(os.path.join(raw_data_path, "publicru-dataset-2020-03-16", "lib_public_document.csv"), index_col="id")

In [81]:
df = df_processed_documents[:100]

In [82]:
df.head()

,content_api_id,title,annotation,body,pages,authors,size,last_modified,issue_id,pages_visual
id,,,,,,,,,,
325,182582609,Эффект слабого рубля,Мы взяли две компании с сопоставимой выручкой ...,<body><p><b>Мы взяли две компании с сопоставим...,"[70, 71]",NaN,40754,2016-11-24 13:15:42.057148+00:00,9,"['70', '71']"
21225,175907949,РОССТАТ,Индексы потребительских цен на товары и услуги...,<body><p><b>Индексы потребительских цен на тов...,[4],NaN,28483,2016-11-24 23:46:59.007647+00:00,599,['4']
21226,175907948,Неотложка с планшетом,ЗДОРОВЬЕ . В системе ЕМИАС зарегистрировались ...,<body><p><b>ЗДОРОВЬЕ</b> </p>\n<p>В системе ЕМ...,[4],Сергей Жуков,30346,2016-11-24 23:46:59.090845+00:00,599,['4']
1638,183704356,ПРОГНОЗЫ НОМЕРА,К 2017-2018 годам ныне растущий агрокомплекс с...,<body><p>К 2017-2018 годам ныне растущий агрок...,[5],NaN,818,2016-11-24 14:16:32.322323+00:00,42,['5']
2,154615279,ТАНЕЦ НЕНАСТОЯЩЕГО ЛЕБЕДЯ,ПЕРЕД НАМИ КАК БУДТО СЦЕНА ИЗ БАЛЕТА: ДВА МИСТ...,<body><p>ПЕРЕД НАМИ КАК БУДТО СЦЕНА ИЗ БАЛЕТА:...,"[60, 61]",NaN,44550,2016-11-24 13:01:38.412930+00:00,1,"['60', '61']"


In [89]:
class TextNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, soup=True):
        self.load_stop_words()
        self.morph = pymorphy2.MorphAnalyzer()
        self.soup = soup
        self.meaning_cache = {}  # Кеш значимых слов.
        self.insignificant_cache = {}  # Кеш незначимых слов.
        self.pos = ['NOUN', 'ADJF', 'ADJS', 'VERB', 'INFN', 'ADVB']
        self.sentence_min_len = 5

    def load_stop_words(self):
        with open(os.path.join(external_data_path, 'russian_stop_words.txt'), 'r') as handle:
            stop_words_ru = [line.rstrip('\n') for line in handle]
        with open(os.path.join(external_data_path, 'english_stop_words.txt'), 'r') as handle:
            stop_words_en = [line.rstrip('\n') for line in handle]
        self.stop_words = set(stop_words_ru + stop_words_en)

    # Фильтруем по части речи и возвращаем только начальную форму.
    def lemmatize(self, tokens, need_pos=False):
        words = []
        for token in tokens:
            # Если токен уже был закеширован, быстро возьмем результат из кэша.
            if token in self.meaning_cache.keys():
                words.append(self.meaning_cache[token])
            elif token in self.insignificant_cache.keys():
                pass
            # Слово еще не встретилось, будем проводить медленный морфологический анализ.
            else:
                result = self.morph.parse(token)
                if result[0].tag.POS != None:
                    if result[0].tag.POS in self.pos:
                        if need_pos:
                            word = result[0].normal_form + "_" + result[0].tag.POS
                        else:
                            word = result[0].normal_form
                        # Отправляем слово в результат, ...
                        words.append(word)
                        # ... и кешируем результат его разбора.
                        self.meaning_cache[token] = word
                    else:
                        self.insignificant_cache[token] = ""
                if 'LATN' in result[0].tag:
                    if need_pos:
                        word = result[0].normal_form + "_" + result[0].tag.POS
                    else:
                        word = result[0].normal_form
                    words.append(word)
                    self.meaning_cache[token] = word

        return words

    def normalize(self, document, exclude_short_sent):
        if self.soup:
            soup = BeautifulSoup(str(document), 'html.parser')
            document = soup.get_text()
        doc_tokens = []
        for sentence in sent_tokenize(document):
            sentence = sentence.replace("«", "").replace("»", "").replace("/", " ")
            sentence = sentence.replace("№", "").replace("-", " ").replace("–", " ").replace(":", " ").replace("/", " ")
            sentence = sentence.replace("ё", "е")
            tokens = word_tokenize(sentence)
            if exclude_short_sent and len(tokens) < self.sentence_min_len:
                tokens = []
                continue
            tokens = [i.lower() for i in tokens if i not in string.punctuation]
            tokens = [i for i in tokens if i not in self.stop_words]
            doc_tokens.extend(self.lemmatize(tokens, False))
        return doc_tokens

    def fit_transform(self, documents, exclude_short_sent=True):
        result = []
        for doc in tqdm(documents):
            result.append(self.normalize(doc, exclude_short_sent))
        return result

    def save(self, filename):
        try:
            with open(filename, 'wb') as handle:
                pickle.dump([self.meaning_cache, self.insignificant_cache], handle, protocol=pickle.HIGHEST_PROTOCOL)
        except IOError:
            logging.info("File not accessible")

    def load(self, filename):
        try:
            with open(filename, 'rb') as handle:
                self.meaning_cache, self.insignificant_cache = pickle.load(handle)
        except IOError:
            logging.info("File not accessible")


In [84]:
text = "Красивая мама красиво мыла раму"
m = Mystem()
lemmas = m.lemmatize(text)

print("lemmas:", ''.join(lemmas))
print("full info:", json.dumps(m.analyze(text), ensure_ascii=False))


Installing mystem to /home/science/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


lemmas: красивый мама красиво мыть рама

full info: [{"analysis": [{"lex": "красивый", "wt": 1, "gr": "A=им,ед,полн,жен"}], "text": "Красивая"}, {"text": " "}, {"analysis": [{"lex": "мама", "wt": 1, "gr": "S,жен,од=им,ед"}], "text": "мама"}, {"text": " "}, {"analysis": [{"lex": "красиво", "wt": 0.8149252476, "gr": "ADV="}], "text": "красиво"}, {"text": " "}, {"analysis": [{"lex": "мыть", "wt": 0.441520999, "gr": "V,несов,пе=прош,ед,изъяв,жен"}], "text": "мыла"}, {"text": " "}, {"analysis": [{"lex": "рама", "wt": 0.9993591156, "gr": "S,жен,неод=вин,ед"}], "text": "раму"}, {"text": "\n"}]


In [85]:
text = "У нас нет мыла"
lemmas = m.lemmatize(text)
print("lemmas:", ''.join(lemmas))
print("full info:", json.dumps(m.analyze(text), ensure_ascii=False))

lemmas: у мы нет мыло

full info: [{"analysis": [{"lex": "у", "wt": 0.9993940324, "gr": "PR="}], "text": "У"}, {"text": " "}, {"analysis": [{"lex": "мы", "wt": 1, "gr": "SPRO,мн,1-л=(пр|вин|род)"}], "text": "нас"}, {"text": " "}, {"analysis": [{"lex": "нет", "wt": 0.464233437, "gr": "ADV,прдк="}], "text": "нет"}, {"text": " "}, {"analysis": [{"lex": "мыло", "wt": 0.558479001, "gr": "S,сред,неод=(вин,мн|род,ед|им,мн)"}], "text": "мыла"}, {"text": "\n"}]


In [86]:
text = "You should never modify something you are iterating over. This is not guaranteed to work in all cases. Depending on the data types, the iterator returns a copy and not a view, and writing to it will have no effect."

lemmas = m.lemmatize(text)
print("lemmas:", ''.join(lemmas))
print("full info:", json.dumps(m.analyze(text), ensure_ascii=False))


lemmas: You should never modify something you are iterating over. This is not guaranteed to work in all cases. Depending on the data types, the iterator returns a copy and not a view, and writing to it will have no effect.

full info: [{"analysis": [], "text": "You"}, {"text": " "}, {"analysis": [], "text": "should"}, {"text": " "}, {"analysis": [], "text": "never"}, {"text": " "}, {"analysis": [], "text": "modify"}, {"text": " "}, {"analysis": [], "text": "something"}, {"text": " "}, {"analysis": [], "text": "you"}, {"text": " "}, {"analysis": [], "text": "are"}, {"text": " "}, {"analysis": [], "text": "iterating"}, {"text": " "}, {"analysis": [], "text": "over"}, {"text": ". "}, {"analysis": [], "text": "This"}, {"text": " "}, {"analysis": [], "text": "is"}, {"text": " "}, {"analysis": [], "text": "not"}, {"text": " "}, {"analysis": [], "text": "guaranteed"}, {"text": " "}, {"analysis": [], "text": "to"}, {"text": " "}, {"analysis": [], "text": "work"}, {"text": " "}, {"analysis": []

In [92]:
morph = pymorphy2.MorphAnalyzer()

In [103]:
text = "У нас нет мыла"
for sentence in sent_tokenize(text):
    tokens = word_tokenize(sentence)
    for token in tokens:
        pprint(morph.parse(token))

[Parse(word='у', tag=OpencorporaTag('PREP'), normal_form='у', score=0.995135, methods_stack=((<DictionaryAnalyzer>, 'у', 24, 0),)),
 Parse(word='у', tag=OpencorporaTag('INTJ'), normal_form='у', score=0.004864, methods_stack=((<DictionaryAnalyzer>, 'у', 21, 0),)),
 Parse(word='у', tag=OpencorporaTag('NOUN,anim,masc,Sgtm,Name,Fixd,Abbr,Init sing,nomn'), normal_form='у', score=0.0, methods_stack=((<AbbreviatedFirstNameAnalyzer>, 'У'),)),
 Parse(word='у', tag=OpencorporaTag('NOUN,anim,masc,Sgtm,Name,Fixd,Abbr,Init sing,gent'), normal_form='у', score=0.0, methods_stack=((<AbbreviatedFirstNameAnalyzer>, 'У'),)),
 Parse(word='у', tag=OpencorporaTag('NOUN,anim,masc,Sgtm,Name,Fixd,Abbr,Init sing,datv'), normal_form='у', score=0.0, methods_stack=((<AbbreviatedFirstNameAnalyzer>, 'У'),)),
 Parse(word='у', tag=OpencorporaTag('NOUN,anim,masc,Sgtm,Name,Fixd,Abbr,Init sing,accs'), normal_form='у', score=0.0, methods_stack=((<AbbreviatedFirstNameAnalyzer>, 'У'),)),
 Parse(word='у', tag=OpencorporaTag(

In [105]:
text = "Красивая мама красиво мыла раму"
for sentence in sent_tokenize(text):
    tokens = word_tokenize(sentence)
    for token in tokens:
        pprint(morph.parse(token))

[Parse(word='красивая', tag=OpencorporaTag('ADJF,Qual femn,sing,nomn'), normal_form='красивый', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'красивая', 1800, 7),))]
[Parse(word='мама', tag=OpencorporaTag('NOUN,anim,femn sing,nomn'), normal_form='мама', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'мама', 1907, 0),))]
[Parse(word='красиво', tag=OpencorporaTag('ADVB'), normal_form='красиво', score=0.8, methods_stack=((<DictionaryAnalyzer>, 'красиво', 3, 0),)),
 Parse(word='красиво', tag=OpencorporaTag('ADJS,Qual neut,sing'), normal_form='красивый', score=0.2, methods_stack=((<DictionaryAnalyzer>, 'красиво', 1800, 56),))]
[Parse(word='мыла', tag=OpencorporaTag('NOUN,inan,neut sing,gent'), normal_form='мыло', score=0.333333, methods_stack=((<DictionaryAnalyzer>, 'мыла', 54, 1),)),
 Parse(word='мыла', tag=OpencorporaTag('VERB,impf,tran femn,sing,past,indc'), normal_form='мыть', score=0.333333, methods_stack=((<DictionaryAnalyzer>, 'мыла', 1813, 8),)),
 Parse(word='мыла', tag=Openco